##第5章: 係り受け解析
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

まずは指定のデータをダウンロードします。
Google Colaboratoryのセル上で下記のコマンドを実行すると、カレントディレクトリに対象のファイルがダウンロードされます。

In [1]:
!wget https://nlp100.github.io/data/ai.ja.zip
!unzip ai.ja.zip

--2021-01-15 04:09:09--  https://nlp100.github.io/data/ai.ja.zip
Resolving nlp100.github.io (nlp100.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17516 (17K) [application/zip]
Saving to: ‘ai.ja.zip’

ai.ja.zip           100%[===================>]  17.11K  --.-KB/s    in 0s      

2021-01-15 04:09:09 (128 MB/s) - ‘ai.ja.zip’ saved [17516/17516]

Archive:  ai.ja.zip
  inflating: ai.ja.txt               
  inflating: readme.ai.ja.md         


CaboChaおよびCaboChaの実行に必要なMeCabとCRF++をインストールします。

In [2]:
# MeCabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
0 upgraded, 8 newly installed, 0 to remove and 16 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 277 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic-utf8 all 7.0-20130310-4 [16.2 MB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd6

In [3]:
# CRF++のソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7QVR6VXJ5dWExSTQ"
FILE_NAME = "crfpp.tar.gz"
!wget 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O $FILE_NAME
!tar xvf crfpp.tar.gz
%cd CRF++-0.58
!./configure && make && make install && ldconfig
%cd ..

--2021-01-15 04:09:43--  https://docs.google.com/uc?export=download&id=0B4y35FiV1wh7QVR6VXJ5dWExSTQ
Resolving docs.google.com (docs.google.com)... 64.233.189.138, 64.233.189.100, 64.233.189.139, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-74-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/alhgo7dmav72l3gi7eb1rsqqrvl2onl0/1610683725000/13553212398903315502/*/0B4y35FiV1wh7QVR6VXJ5dWExSTQ?e=download [following]
--2021-01-15 04:09:44--  https://doc-08-74-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/alhgo7dmav72l3gi7eb1rsqqrvl2onl0/1610683725000/13553212398903315502/*/0B4y35FiV1wh7QVR6VXJ5dWExSTQ?e=download
Resolving doc-08-74-docs.googleusercontent.com (doc-08-74-docs.googleusercontent.com)... 108.177.125.132, 2404:6800:4008:c01::84
Connecting to doc-08-74-docs.googleusercontent.com (doc-08-74-docs.googleuser

In [4]:
# CaboChaのソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7SDd1Q1dUQkZQaUU"
FILE_NAME = "cabocha-0.69.tar.bz2"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt
!tar -xvf cabocha-0.69.tar.bz2
%cd cabocha-0.69
!./configure -with-charset=utf-8 && make && make check && make install && ldconfig
%cd ..

--2021-01-15 04:10:22--  https://docs.google.com/uc?export=download&confirm=Q3zc&id=0B4y35FiV1wh7SDd1Q1dUQkZQaUU
Resolving docs.google.com (docs.google.com)... 64.233.188.102, 64.233.188.113, 64.233.188.101, ...
Connecting to docs.google.com (docs.google.com)|64.233.188.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-90-docs.googleusercontent.com/docs/securesc/ebtgm6tu5r5q55euvuah615c7n16ppcj/bra73v3koo8luap2ouc03eg78e2l9hkl/1610683800000/13553212398903315502/14599972441048925356Z/0B4y35FiV1wh7SDd1Q1dUQkZQaUU?e=download [following]
--2021-01-15 04:10:22--  https://doc-0o-90-docs.googleusercontent.com/docs/securesc/ebtgm6tu5r5q55euvuah615c7n16ppcj/bra73v3koo8luap2ouc03eg78e2l9hkl/1610683800000/13553212398903315502/14599972441048925356Z/0B4y35FiV1wh7SDd1Q1dUQkZQaUU?e=download
Resolving doc-0o-90-docs.googleusercontent.com (doc-0o-90-docs.googleusercontent.com)... 108.177.125.132, 2404:6800:4008:c01::84
Connecting to doc-0o-90-

係り受け解析を行います。
以下のコマンドを実行することにより、ai.ja.txtを係り受け解析した結果が、ai.ja.txt.parsedとして出力されます。

In [5]:
!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt 

出力結果を確認します。

In [6]:
# 行数の確認
!wc -l ./ai.ja.txt.parsed

11744 ./ai.ja.txt.parsed


In [7]:
# 先頭15行の確認
!head -15 ./ai.ja.txt.parsed

* 0 -1D 1/1 0.000000
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
EOS
EOS
* 0 17D 1/1 0.388993
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
* 1 17D 2/3 0.613549
（	記号,括弧開,*,*,*,*,（,（,（
じん	名詞,一般,*,*,*,*,じん,ジン,ジン
こうち	名詞,一般,*,*,*,*,こうち,コウチ,コーチ
のう	助詞,終助詞,*,*,*,*,のう,ノウ,ノー
、	記号,読点,*,*,*,*,、,、,、
、	記号,読点,*,*,*,*,、,、,、


####40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [8]:
class Morph:
  def __init__(self, morph):
    surface, attr = morph.split('\t')
    attr = attr.split(',')
    self.surface = surface
    self.base = attr[6]
    self.pos = attr[0]
    self.pos1 = attr[1]

In [9]:
filename = './ai.ja.txt.parsed'

sentences = []
morphs = []
with open(filename, mode='r') as f:
  for line in f:
    if line[0] == '*':  # 係り受け関係を表す行：スキップ
      continue
    elif line != 'EOS\n':  # 文末以外：Morphを適用し形態素リストに追加
      morphs.append(Morph(line))
    else:  # 文末：形態素リストを文リストに追加
      sentences.append(morphs)
      morphs = []

# 確認
for m in sentences[2]:
  print(vars(m))

{'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '一般'}
{'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '一般'}
{'surface': '（', 'base': '（', 'pos': '記号', 'pos1': '括弧開'}
{'surface': 'じん', 'base': 'じん', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'こうち', 'base': 'こうち', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'のう', 'base': 'のう', 'pos': '助詞', 'pos1': '終助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': 'AI', 'base': '*\n', 'pos': '名詞', 'pos1': '一般'}
{'surface': '〈', 'base': '〈', 'pos': '記号', 'pos1': '括弧開'}
{'surface': 'エーアイ', 'base': '*\n', 'pos': '名詞', 'pos1': '固有名詞'}
{'surface': '〉', 'base': '〉', 'pos': '記号', 'pos1': '括弧閉'}
{'surface': '）', 'base': '）', 'pos': '記号', 'pos1': '括弧閉'}
{'surface': 'と', 'base': 'と', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '「', 'base': '「', 'pos': '記号', 'pos1': '括弧開'}
{

####41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

文章は文(sentence)オブジェクトのリストで表され、文オブジェクトは文節(chunk)オブジェクトのリストを要素に持ち、文節オブジェクトは形態素(morph)オブジェクトのリストを要素に持つ階層構造を考え、ここでは指定のクラスChunkに加え、Sentenceを実装しています。
なお、Chunkオブジェクトの要素である係り元文節インデックス番号のリスト(srcs)の作成には1文のすべての文節情報を必要とするため、Sentenceオブジェクトの初期化時に作成しています。

In [44]:
class Chunk():
  def __init__(self, morphs, dst):
    self.morphs = morphs
    self.dst = dst
    self.srcs = []


class Sentence():
  def __init__(self, chunks):
    self.chunks = chunks
    for i, chunk in enumerate(self.chunks):
      if chunk.dst != -1:
        self.chunks[chunk.dst].srcs.append(i)

In [19]:
class Morph:
    def __init__(self, dc):
        self.surface = dc['surface']
        self.base = dc['base']
        self.pos = dc['pos']
        self.pos1 = dc['pos1']


class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs    # 形態素（Morphオブジェクト）のリスト
        self.dst = dst          # 係り先文節インデックス番号
        self.srcs = []          # 係り元文節インデックス番号のリスト


def parse_cabocha(block):
    def check_create_chunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            res.append(c)
            tmp = []
        return tmp

    res = []
    tmp = []
    dst = None
    for line in block.split('\n'):
        if line == '':
            tmp = check_create_chunk(tmp)
        elif line[0] == '*':
            dst = line.split(' ')[2].rstrip('D')
            tmp = check_create_chunk(tmp)
        else:
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            lineDict = {
                'surface': surface,
                'base': attr[6],
                'pos': attr[0],
                'pos1': attr[1]
            }
            tmp.append(Morph(lineDict))

    for i, r in enumerate(res):
        res[int(r.dst)].srcs.append(i)
    return res

In [24]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]
for m in blocks[7]:
    print([mo.surface for mo in m.morphs], m.dst, m.srcs)

['一方', '、'] 2 []
['計算', '知能'] 12 []
['（', 'CI', '）', 'は'] 4 [0]
['開発', 'や'] 5 []
['学習', 'を'] 6 [2]
['繰り返す'] 8 [3]
['こと', 'を'] 8 [4]
['基本', 'と'] 9 []
['し', 'て', 'いる'] 12 [5, 6]
['（', '例えば', '、'] 12 [7]
['パラメータ', '調整', '、'] 12 []
['コネクショニズム', 'の'] 21 []
['システム', '）', '。'] 16 [1, 8, 9, 10]
['学習', 'は'] 15 []
['経験', 'に'] 16 []
['基づく'] 21 [13]
['手法', 'で', 'あり', '、'] 18 [12, 14]
['非', '記号', '的', 'AI', '、'] 20 []
['美しく', 'ない'] 20 [16]
['AI', '、'] 21 []
['ソフトコンピューティング', 'と'] 26 [17, 18]
['関係', 'し', 'て', 'いる', '。'] 23 [11, 15, 19]
['その'] 26 []
['手法', 'として', 'は', '、'] 25 [21]
['以下', 'の'] 26 []
['もの', 'が'] -1 [23]
['ある', '。'] -1 [20, 22, 24, 25, 26]


####42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [27]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if int(m.dst) > -1:
            print(''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs]),
                  ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs]), sep='\t')

人工知能	語
じんこうちのう	エーアイとは
AI	語
エーアイとは	という
計算	道具を
という	道具を
概念と	という
コンピュータ	道具を
という	用いて
道具を	研究する
用いて	研究する
知能を	計算機科学
研究する	の
計算機科学	一分野を
の	指す
一分野を	語
指す	研究分野とも
語	推論
言語の	推論
理解や	問題解決などの
推論	知的行動を
問題解決などの	代わって
知的行動を	代わって
人間に	行わせる
代わって	行わせる
コンピューターに	技術または
行わせる	研究分野とも
技術または	コンピュータによる
計算機	情報処理システムの
コンピュータによる	情報処理システムの
知的な	実現に関する
情報処理システムの	実現に関する
設計や	研究分野とも
実現に関する	される
日本大百科全書(ニッポニカ)』の	述べている
解説で	佐藤理史は
情報工学者通信工学者の	述べている
佐藤理史は	述べている
人間の	実現する
知的能力を	実現する
コンピュータ上で	技術ソフトウェアコンピュータシステム
実現する	技術ソフトウェアコンピュータシステム
様々な	ある
技術ソフトウェアコンピュータシステム	ある
応用例は	機械翻訳かな漢字変換構文解析等
自然言語処理	専門家の
機械翻訳かな漢字変換構文解析等	推論判断を
専門家の	模倣する
推論判断を	エキスパートシステム
模倣する	画像認識等が
エキスパートシステム	解析して
画像データを	検出抽出したりする
解析して	パターンを
特定の	検出抽出したりする
パターンを	画像認識等が
検出抽出したりする	ある
画像認識等が	命名された
ある	命名された
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	使われている
命名された	使われている
現在では	用いた
記号処理を	知能の
用いた	記述を
知能の	する
記述を	する
主体と	研究での
する	研究での
情報処理や	アプローチという
研究での	意味あいでも
アプローチという	使われている
意味あいでも	ある
使われている	制御システムや
家庭用電気機械器具の	思考ルーチンも
制御システムや	思考ルーチンも
ゲームソフトの	呼ばれる
思考ルーチンも	呼ばれる
こう	ことも
呼ばれる	ある
プログラミング言語による	カウンセラーを
と

####43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [28]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if int(m.dst) > -1:
            pre_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
            pre_pos = [mo.pos for mo in m.morphs]
            post_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs])
            post_pos = [mo.pos for mo in b[int(m.dst)].morphs]
            if '名詞' in pre_pos and '動詞' in post_pos:
                print(pre_text, post_text, sep='\t')

道具を	研究する
問題解決などの	代わって
知的行動を	代わって
人間に	行わせる
実現に関する	される
日本大百科全書(ニッポニカ)』の	述べている
情報工学者通信工学者の	述べている
佐藤理史は	述べている
人間の	実現する
知的能力を	実現する
様々な	ある
技術ソフトウェアコンピュータシステム	ある
専門家の	模倣する
エキスパートシステム	解析して
画像データを	検出抽出したりする
特定の	検出抽出したりする
検出抽出したりする	ある
画像認識等が	命名された
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	使われている
命名された	使われている
現在では	用いた
知能の	する
記述を	する
アプローチという	使われている
意味あいでも	ある
ゲームソフトの	呼ばれる
思考ルーチンも	呼ばれる
模倣した	出されるが
プログラム	出されるが
人工無脳が	出されるが
引き合いに	困難視されている
専門家の	させようという
研究情報処理システムの	困難視されている
実現は	持つ
人間が	持つ
常識の	なり
記述が	なり
問題と	困難視されている
実用への	困難視されている
利用が	ある
実現への	知られているが
ニューラルネットワークなどのような	知られているが
アプローチも	ある
(GoodOldFashionedAI)との	ある
記述の	ある
記号的明示性に	集めた
その後	集めた
サポートベクターマシンが	集めた
注目を	ある
自らの	行う
経験を	行う
元に	行う
強化学習という	ある
レイカーツワイルという	実装するという
言葉通り	表現し
知性を	表現し
機械的に	実装するという
ビッグデータの	行った
一過性の	超えて
流行を	浸透して
社会に	行った
浸透して	なった
2016年から	導入した
2017年にかけて	導入した
世界トップクラスの	破り
プレイヤーも	なった
麻雀では	到達するなど
MicrosoftSuphx(SuperPhoenix)」が	到達するなど
AIとして	到達するなど
十段に	なった
時代の	なった
第２次人工知能ブームでの	呼ばれ
人工知能は	呼ばれ
機械学習と	ある
以下のような	ある
開発や	繰り返す
ことを	している
コネクショニズムの	関係している
学習は	基づく
AI	関係し

####44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

係り元と係り先の文節のペアを作成し、pydotのgraph_from_edgesに渡すことで、グラフを作成しています。
なお、表層形そのままでは1文内に同じ文字列の文節が複数回出てきた場合に区別できないため、末尾にIDを付与して表示しています。



In [29]:
# 日本語表示用フォントのインストール
!apt install fonts-ipafont-gothic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-ipafont-mincho
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 16 not upgraded.
Need to get 8,251 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-gothic all 00303-18ubuntu1 [3,526 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-mincho all 00303-18ubuntu1 [4,725 kB]
Fetched 8,251 kB in 3s (2,865 kB/s)
Selecting previously unselected package fonts-ipafont-gothic.
(Reading database ... 145661 files and directories currently installed.)
Preparing to unpack .../fonts-ipafont-gothic_00303-18ubuntu1_all.deb ...
Unpacking fonts-ipafont-gothic (00303-18ubuntu1) ...
Selecting previously unselected package fonts-ip

In [31]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

pairs = []
target = blocks[7]
for m in target:
    if int(m.dst) > -1:
        pre_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
        post_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in target[int(m.dst)].morphs])
        pairs.append([pre_text, post_text])

print(pairs)
g = pydot.graph_from_edges(pairs)
g.write_png('./ans44.png', prog='dot')

[['一方', 'CIは'], ['計算知能', 'システム'], ['CIは', '学習を'], ['開発や', '繰り返す'], ['学習を', 'ことを'], ['繰り返す', 'している'], ['ことを', 'している'], ['基本と', '例えば'], ['している', 'システム'], ['例えば', 'システム'], ['パラメータ調整', 'システム'], ['コネクショニズムの', '関係している'], ['システム', '手法であり'], ['学習は', '基づく'], ['経験に', '手法であり'], ['基づく', '関係している'], ['手法であり', '美しくない'], ['非記号的AI', 'ソフトコンピューティングと'], ['美しくない', 'ソフトコンピューティングと'], ['AI', '関係している'], ['ソフトコンピューティングと', 'ある'], ['関係している', '手法としては'], ['その', 'ある'], ['手法としては', 'ものが'], ['以下の', 'ある']]


####45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

* 動詞を含む文節において，最左の動詞の基本形を述語とする
* 述語に係る助詞を格とする
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．
* コーパス中で頻出する述語と格パターンの組み合わせ
* 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [49]:
with open('./ans45.txt', 'w') as f:
  for sentence in sentences:
    for chunk in sentence.chunks:
      for morph in chunk.morphs:
        if morph.pos == '動詞':  # chunkの左から順番に動詞を探す
          cases = []
          for src in chunk.srcs:  # 見つけた動詞の係り元chunkから助詞を探す
            cases = cases + [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == '助詞']
          if len(cases) > 0:  # 助詞が見つかった場合は重複除去後辞書順にソートして出力
            cases = sorted(list(set(cases)))
            line = '{}\t{}'.format(morph.base, ' '.join(cases))
            print(line, file=f)
          break

In [42]:
class Morph:
    def __init__(self, dc):
        self.surface = dc['surface']
        self.base = dc['base']
        self.pos = dc['pos']
        self.pos1 = dc['pos1']


class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs    # 形態素（Morphオブジェクト）のリスト
        self.dst = dst          # 係り先文節インデックス番号
        self.srcs = []          # 係り元文節インデックス番号のリスト


def parse_cabocha(block):
    def check_create_chunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            res.append(c)
            tmp = []
        return tmp

    res = []
    tmp = []
    dst = None
    for line in block.split('\n'):
        if line == '':
            tmp = check_create_chunk(tmp)
        elif line[0] == '*':
            dst = line.split(' ')[2].rstrip('D')
            tmp = check_create_chunk(tmp)
        else:
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            lineDict = {
                'surface': surface,
                'base': attr[6],
                'pos': attr[0],
                'pos1': attr[1]
            }
            tmp.append(Morph(lineDict))

    for i, r in enumerate(res):
        res[int(r.dst)].srcs.append(i)
    return res

In [47]:
filename = './ans45.txt'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if len(m.srcs) > 0:
            pre_morphs = [b[int(s)].morphs for s in m.srcs]
            pre_morphs = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in pre_morphs]
            pre_surface = [[p.surface for p in pm] for pm in pre_morphs]
            pre_surface = list(filter(lambda x: x != [], pre_surface))
            pre_surface = [p[0] for p in pre_surface]
            post_base = [mo.base for mo in m.morphs]
            post_pos = [mo.pos for mo in m.morphs]
            if len(pre_surface) > 0 and '動詞' in post_pos:
                print(post_base[0], ' '.join(pre_surface), sep='\t')

In [48]:
# 確認
!cat ./ans45.txt | sort | uniq -c | sort -nr | head -n 10

In [34]:
!cat ./ans45.txt | grep '行う' | sort | uniq -c | sort -nr | head -n 5

In [35]:
!cat ./ans45.txt | grep 'なる' | sort | uniq -c | sort -nr | head -n 5

In [36]:
!cat ./ans45.txt | grep '与える' | sort | uniq -c | sort -nr | head -n 5

####46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

* 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
* 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

In [45]:
with open('./ans46.txt', 'w') as f:
  for sentence in sentences:
    for chunk in sentence.chunks:
      for morph in chunk.morphs:
        if morph.pos == '動詞':  # chunkの左から順番に動詞を探す
          cases = []
          modi_chunks = []
          for src in chunk.srcs:  # 見つけた動詞の係り元chunkから助詞を探す
            case = [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == '助詞']
            if len(case) > 0:  # 助詞を含むchunkの場合は助詞と項を取得
              cases = cases + case
              modi_chunks.append(''.join(morph.surface for morph in sentence.chunks[src].morphs if morph.pos != '記号'))
          if len(cases) > 0:  # 助詞が1つ以上見つかった場合は重複除去後辞書順にソートし、項と合わせて出力
            cases = sorted(list(set(cases)))
            line = '{}\t{}\t{}'.format(morph.base, ' '.join(cases), ' '.join(modi_chunks))
            print(line, file=f)
          break

In [46]:
# 確認
!cat ./ans46.txt | head -n 10